In [129]:
#Notebook description

#This notebook is being used to evaluate the techinical market conditions of a single asset and assess
#the appropriate strategy to take in order to maximize returns.

In [130]:
#take all compuation functions and put them in a separate file

#simplify the date x axis on the percent drawdown chart

#default the zoom range to a comfortable range, and create a dropdown to select the time range for Volatility section

#properly label and annoate the garch models

#Remove the VIX charting, its redudnant now that we have the volatility models

In [131]:
#Load libraries 
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True
logger.propagate = False
# Load libraries
from Quantapp.Plotter import Plotter
from Quantapp.Computation import Computation
from Quantapp.EconomicData import EconomicData

import numpy as np
import json
import os
import pandas as pd
import yfinance as yf
from statsmodels.tsa.stattools import coint
from IPython.display import display
from concurrent.futures import ThreadPoolExecutor
from plotly.subplots import make_subplots
from datetime import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.graph_objects as go
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots

#shut down warnings
import warnings
warnings.filterwarnings("ignore")


qc = Computation()
qp = Plotter()
qe = EconomicData()

def simplify_datetime_index(series):
    """
    Simplifies the DateTime index of a Series to contain only the date (YYYY-MM-DD),
    maintaining it as a DateTimeIndex without timezone information.
    
    Parameters:
        series (pd.Series): The input Series with a DateTimeIndex.
    
    Returns:
        pd.Series: The Series with the DateTime index simplified to YYYY-MM-DD.
    """
    if not isinstance(series.index, pd.DatetimeIndex):
        raise TypeError("The Series index must be a DateTimeIndex.")
    
    # Remove timezone information if present
    if series.index.tz is not None:
        series = series.copy()
        series.index = series.index.tz_convert('UTC').tz_localize(None)
    
    # Normalize the index to remove the time component
    series.index = series.index.normalize()
    
    return series
                

In [132]:
#PARAMETERS

time_frame_week = 7
time_frame_short = 21
time_frame_mid   = 50
time_frame_long = 200

interval = '1d'
period     = '10y'
risk_free_rate = 0.02 / 252  # Annualized risk-free rate divided by trading days

#should be a string or None
ticker_str = 'EWW'
primary_benchmark_str = 'SPY'
secondary_benchmark_str = None



In [133]:
#Load data

ticker = yf.Ticker(ticker_str).history(period=period, interval=interval)
vix = yf.Ticker('^VIX').history(period=period, interval=interval)

# Fetch and simplify benchmark data only if the strings are not None
if primary_benchmark_str is not None:
    primary_benchmark = yf.Ticker(primary_benchmark_str).history(period=period, interval=interval)
    primary_benchmark = simplify_datetime_index(primary_benchmark)
else:
    primary_benchmark = None

if secondary_benchmark_str is not None:
    secondary_benchmark = yf.Ticker(secondary_benchmark_str).history(period=period, interval=interval)
    secondary_benchmark = simplify_datetime_index(secondary_benchmark)
else:
    secondary_benchmark = None

# Simplify ticker and vix
ticker = simplify_datetime_index(ticker)
vix = simplify_datetime_index(vix)

# Align ticker data with primary benchmark if available
if primary_benchmark is not None:
    ticker = ticker[ticker.index.isin(primary_benchmark.index)]

# Calculate ticker returns
ticker_monthly_returns = qc.calculate_returns(ticker, frequency='monthly')
ticker_weekly_returns = qc.calculate_returns(ticker, frequency='weekly')
ticker_daily_returns = qc.calculate_returns(ticker, frequency='daily')

# Calculate primary benchmark returns if it exists
if primary_benchmark is not None:
    primary_benchmark_monthly_returns = qc.calculate_returns(primary_benchmark, frequency='monthly')
    primary_benchmark_weekly_returns = qc.calculate_returns(primary_benchmark, frequency='weekly')
    primary_benchmark_daily_returns = qc.calculate_returns(primary_benchmark, frequency='daily')

# Calculate secondary benchmark returns if it exists
if secondary_benchmark is not None:
    secondary_benchmark_monthly_returns = qc.calculate_returns(secondary_benchmark, frequency='monthly')
    secondary_benchmark_weekly_returns = qc.calculate_returns(secondary_benchmark, frequency='weekly')
    secondary_benchmark_daily_returns = qc.calculate_returns(secondary_benchmark, frequency='daily')

# Calculate VIX returns
vix_monthly_returns = qc.calculate_returns(vix, frequency='monthly')
vix_weekly_returns = qc.calculate_returns(vix, frequency='weekly')
vix_daily_returns = qc.calculate_returns(vix, frequency='daily')


In [134]:
#REGIME CHANGES: Candlestick with RSI and Bollinger Bands

def calculate_percentage_drop(ticker, n=14):
    """
    Calculate the percentage drop from the highest peak in a rolling window.

    Parameters:
    - ticker: pd.DataFrame with 'Close' prices indexed by date.
    - n: Number of days for the rolling window.

    Returns:
    - pd.DataFrame with 'Close', 'HighestHigh', and 'PercentageDrop' columns.
    """
    # Ensure ticker is a DataFrame and has a 'Close' column
    if 'Close' not in ticker.columns:
        raise ValueError("The DataFrame must contain a 'Close' column.")
    
    # Calculate the highest peak in the last n days
    ticker['HighestHigh'] = ticker['Close'].rolling(window=n, min_periods=1).max()
    
    # Calculate the percentage drop from the highest peak
    ticker['PercentageDrop'] = -((ticker['HighestHigh'] - ticker['Close']) / ticker['HighestHigh']) * 100
    
    return ticker

def plot_candlestick(ticker_data, drop_window=14, period='1Y', bollinger_window=21):
    """
    Plots the candlestick chart with Bollinger Bands for the given stock data.
    """
    # Remove weekends/holidays and calculate percentage drop
    ticker_data = ticker_data[ticker_data.index.dayofweek < 5]
    holidays = pd.to_datetime(['2023-01-01', '2023-12-25'])  # Add more holidays as needed
    ticker_data = ticker_data[~ticker_data.index.isin(holidays)]
    ticker_data = calculate_percentage_drop(ticker_data, n=drop_window)
    mean_drop = ticker_data['PercentageDrop'].mean()
    std_drop = ticker_data['PercentageDrop'].std()
  
    # Filter data for the specified period
    period_data = ticker_data.last(period)

    # Define bar colors
    colors = [
        'red' if drop < mean_drop - 0.5 * std_drop
        else 'blue' if drop < mean_drop + 0.25 * std_drop
        else 'green'
        for drop in period_data['PercentageDrop']
    ]

    # Calculate Bollinger Bands
    ma = period_data['Close'].rolling(window=bollinger_window).mean()
    std = period_data['Close'].rolling(window=bollinger_window).std()

    
    bollinger_bands = {}
    for k in [1, 2, 3]:
        bollinger_bands[f'Upper_{k}'] = ma + (std * k)
        bollinger_bands[f'Lower_{k}'] = ma - (std * k)
    bollinger_df = pd.DataFrame(bollinger_bands)
    # Create a single-figure candlestick chart
    fig = go.Figure()

    # Add candlestick data
    for i, color in enumerate(colors):
        fig.add_trace(go.Candlestick(
            x=[period_data.index[i]],
            open=[period_data['Open'].iloc[i]],
            high=[period_data['High'].iloc[i]],
            low=[period_data['Low'].iloc[i]],
            close=[period_data['Close'].iloc[i]],
            increasing_line_color=color,
            decreasing_line_color=color,
            showlegend=False
        ))

    # Add Bollinger Bands
    for k in [1, 2, 3]:
        fig.add_trace(go.Scatter(
            x=period_data.index,
            y=bollinger_df[f'Upper_{k}'],
            mode='lines',
            line=dict(width=1, dash='dash'),
            name=f'Upper Band {k} SD'
        ))
        fig.add_trace(go.Scatter(
            x=period_data.index,
            y=bollinger_df[f'Lower_{k}'],
            mode='lines',
            line=dict(width=1, dash='dash'),
            name=f'Lower Band {k} SD'
        ))

    fig.update_layout(
        title='Candlestick With Bollinger Bands',
        xaxis_title='Date',
        yaxis_title='Price',
        height=900,
        template='plotly_dark',
        yaxis=dict(autorange=True, fixedrange=False),
        xaxis=dict(
            rangeslider=dict(visible=False),
            tickangle=-45,
            showgrid=True,
            zeroline=False
        )
    )

    fig.show()

plot_candlestick(ticker, period='2y', bollinger_window=50)
plot_candlestick(ticker, period='5y', bollinger_window=200)


In [135]:
#REGIME CHANGES: Percentage Drop from Highest Peak

# this code block visualizes the regime changes of the SPY ETF
# the regime changes tell us when the market is in a bull or bear market thus telling us the conditions
# that tell us when to buy or sell. 

# for example:
# if the market is in a bear market, we should be selling or shorting assets
# if the market is in a bull market, we should be buying or longing assets
# if the market is in a neutral market, we should be holding, not trading,or entering neutral positions

# the regime changes tell us to exclusively enter positions according to the market conditions and no morre
# it also tells us when to cut losses
n = int(252 / 2)

# Plot drawdowns for the main ticker
ticker_drawdowns = qc.calculate_percentage_drop(ticker)
qp.plot_percentage_drop(ticker_drawdowns, n=n, title=f'{ticker_str} Percentage Drop from Highest Peak')
'''
# Conditionally plot primary benchmark if available
if primary_benchmark is not None:
    primary_drawdowns = qc.calculate_percentage_drop(primary_benchmark)
    qp.plot_percentage_drop(primary_drawdowns, n=n, title=f'Benchmark 1: {primary_benchmark_str} Percentage Drop from Highest Peak')

# Conditionally plot secondary benchmark if available
if secondary_benchmark is not None:
    secondary_drawdowns = qc.calculate_percentage_drop(secondary_benchmark)
    qp.plot_percentage_drop(secondary_drawdowns, n=n, title=f'Benchmark 2: {secondary_benchmark_str} Percentage Drop from Highest Peak')
'''

"\n# Conditionally plot primary benchmark if available\nif primary_benchmark is not None:\n    primary_drawdowns = qc.calculate_percentage_drop(primary_benchmark)\n    qp.plot_percentage_drop(primary_drawdowns, n=n, title=f'Benchmark 1: {primary_benchmark_str} Percentage Drop from Highest Peak')\n\n# Conditionally plot secondary benchmark if available\nif secondary_benchmark is not None:\n    secondary_drawdowns = qc.calculate_percentage_drop(secondary_benchmark)\n    qp.plot_percentage_drop(secondary_drawdowns, n=n, title=f'Benchmark 2: {secondary_benchmark_str} Percentage Drop from Highest Peak')\n"

In [136]:
#REGIME CHANGES: Markov Switching Model
'''
#KEEP in mind that the Markov Switching Model is highly sensitive to large outliers in the data, meaning the models predictions
#can be unreliable during times of extreme market volatility.
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression

def get_regime_probs(data, k_regimes=3):
    model = MarkovRegression(data["Log_Returns"], k_regimes=k_regimes, trend="c", switching_variance=True)
    result = model.fit()
    return result.smoothed_marginal_probabilities, result

def plot_regimes(data, regime_probs, title='Markov Switching Model Regime Detection'):
    fig = make_subplots(
        rows=3, cols=1, shared_xaxes=True,
        subplot_titles=["Low Volatility", "Mild Volatility", "High Volatility"]
    )

    def add_hline(y_val, row_num):
        fig.add_trace(
            go.Scatter(
                x=[data.index[0], data.index[-1]],
                y=[y_val, y_val],
                mode='lines',
                line=dict(color='black', dash='dash'),
                showlegend=False
            ),
            row=row_num, col=1
        )

    # Low volatility
    fig.add_trace(go.Scatter(x=data.index, y=regime_probs[0], line=dict(color='green'), name='Low Volatility'),
                  row=1, col=1)
    add_hline(0.25, 1)
    add_hline(0.75, 1)

    # Mild volatility
    fig.add_trace(go.Scatter(x=data.index, y=regime_probs[1], line=dict(color='orange'), name='Mild Volatility'),
                  row=2, col=1)
    add_hline(0.25, 2)
    add_hline(0.75, 2)

    # High volatility
    fig.add_trace(go.Scatter(x=data.index, y=regime_probs[2], line=dict(color='red'), name='High Volatility'),
                  row=3, col=1)
    add_hline(0.25, 3)
    add_hline(0.75, 3)

    fig.update_layout(
        title=title,
        template="plotly_white",
        hovermode='x unified',
        height=800
    )
    fig.show()

# Example usage:
symbol = ticker_str

# First step: Get data for the main symbol
data_symbol = yf.download(symbol, period=period, interval=interval)
data_symbol["Log_Returns"] = np.log(data_symbol["Close"] / data_symbol["Close"].shift(1))
data_symbol.dropna(inplace=True)

# Calculate and plot regimes for main symbol
regime_probs_symbol, result_symbol = get_regime_probs(data_symbol, k_regimes=3)
plot_regimes(data_symbol, regime_probs_symbol,title='Market Regime Detection for ' + symbol)

# Only process primary benchmark if it exists
if primary_benchmark_str is not None:
    data_primary = yf.download(primary_benchmark_str, period=period, interval=interval)
    data_primary["Log_Returns"] = np.log(data_primary["Close"] / data_primary["Close"].shift(1))
    data_primary.dropna(inplace=True)
    
    regime_probs_primary, result_primary = get_regime_probs(data_primary, k_regimes=3)
    plot_regimes(data_primary, regime_probs_primary, title='Market Regime Detection for ' + primary_benchmark_str)

# Only process secondary benchmark if it exists
if secondary_benchmark_str is not None:
    data_secondary = yf.download(secondary_benchmark_str, period=period, interval=interval)
    data_secondary["Log_Returns"] = np.log(data_secondary["Close"] / data_secondary["Close"].shift(1))
    data_secondary.dropna(inplace=True)
    
    regime_probs_secondary, result_secondary = get_regime_probs(data_secondary, k_regimes=3)
    plot_regimes(data_secondary, regime_probs_secondary, title='Market Regime Detection for ' + secondary_benchmark_str)
'''

'\n#KEEP in mind that the Markov Switching Model is highly sensitive to large outliers in the data, meaning the models predictions\n#can be unreliable during times of extreme market volatility.\nimport yfinance as yf\nimport numpy as np\nimport plotly.graph_objects as go\nfrom plotly.subplots import make_subplots\nfrom statsmodels.tsa.regime_switching.markov_regression import MarkovRegression\n\ndef get_regime_probs(data, k_regimes=3):\n    model = MarkovRegression(data["Log_Returns"], k_regimes=k_regimes, trend="c", switching_variance=True)\n    result = model.fit()\n    return result.smoothed_marginal_probabilities, result\n\ndef plot_regimes(data, regime_probs, title=\'Markov Switching Model Regime Detection\'):\n    fig = make_subplots(\n        rows=3, cols=1, shared_xaxes=True,\n        subplot_titles=["Low Volatility", "Mild Volatility", "High Volatility"]\n    )\n\n    def add_hline(y_val, row_num):\n        fig.add_trace(\n            go.Scatter(\n                x=[data.index

In [137]:
#VOLATILITY FORECASTING: ARCH Models
'''
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (40,14)
from arch import arch_model
import yfinance as yf


returns = yf.Ticker(ticker_str).history(period=period, interval=interval)['Close'].pct_change().dropna()

# Calculate primary benchmark returns if available
if primary_benchmark_str is not None:
    primary_benchmark_returns = yf.Ticker(primary_benchmark_str).history(period=period, interval=interval)['Close'].pct_change().dropna()
else:
    primary_benchmark_returns = None

# Calculate secondary benchmark returns if available
if secondary_benchmark_str is not None:
    secondary_benchmark_returns = yf.Ticker(secondary_benchmark_str).history(period=period, interval=interval)['Close'].pct_change().dropna()
else:
    secondary_benchmark_returns = None
    
ticker_arch_models = {}

# Define ARCH models for the main ticker
ticker_arch_models['garch']      = arch_model(returns)
ticker_arch_models['egarch']     = arch_model(returns, vol='EGARCH')
ticker_arch_models['gjrgarch']   = arch_model(returns, p=1, o=1, q=1)
ticker_arch_models['tgarch']     = arch_model(returns, p=1, o=1, q=1, power=1.0)
ticker_arch_models['aparch']     = arch_model(returns, p=1, o=1, q=1, vol='APARCH')
ticker_arch_models['figarch']    = arch_model(returns, p=1, o=1, q=1, vol='FIGARCH')

# Define ARCH models for the primary benchmark if it exists
primary_benchmark_arch_models = {}
if primary_benchmark_returns is not None:
    primary_benchmark_arch_models['garch']      = arch_model(primary_benchmark_returns)
    primary_benchmark_arch_models['egarch']     = arch_model(primary_benchmark_returns, vol='EGARCH')
    primary_benchmark_arch_models['gjrgarch']   = arch_model(primary_benchmark_returns, p=1, o=1, q=1)
    primary_benchmark_arch_models['tgarch']     = arch_model(primary_benchmark_returns, p=1, o=1, q=1, power=1.0)
    primary_benchmark_arch_models['aparch']     = arch_model(primary_benchmark_returns, p=1, o=1, q=1, vol='APARCH')
    primary_benchmark_arch_models['figarch']    = arch_model(primary_benchmark_returns, p=1, o=1, q=1, vol='FIGARCH')

# Define ARCH models for the secondary benchmark if it exists
secondary_benchmark_arch_models = {}
if secondary_benchmark_returns is not None:
    secondary_benchmark_arch_models['garch']      = arch_model(secondary_benchmark_returns)
    secondary_benchmark_arch_models['egarch']     = arch_model(secondary_benchmark_returns, vol='EGARCH')
    secondary_benchmark_arch_models['gjrgarch']   = arch_model(secondary_benchmark_returns, p=1, o=1, q=1)
    secondary_benchmark_arch_models['tgarch']     = arch_model(secondary_benchmark_returns, p=1, o=1, q=1, power=1.0)
    secondary_benchmark_arch_models['aparch']     = arch_model(secondary_benchmark_returns, p=1, o=1, q=1, vol='APARCH')
    secondary_benchmark_arch_models['figarch']    = arch_model(secondary_benchmark_returns, p=1, o=1, q=1, vol='FIGARCH')

#square returns
squared_returns = returns**2

ticker_arch_model_results = {}
primary_benchmark_arch_model_results = {}
secondary_benchmark_arch_model_results = {}

# Fit ARCH models for the main ticker
for key, model in ticker_arch_models.items():
    try:
        fitted_model = model.fit(disp="off")
        ticker_arch_model_results[key] = fitted_model.conditional_volatility
    except Exception as e:
        print(f"Error fitting {key} model for ticker {ticker_str}: {e}")

# Fit ARCH models for the primary benchmark if it exists
for key, model in primary_benchmark_arch_models.items():
    try:
        fitted_model = model.fit(disp="off")
        primary_benchmark_arch_model_results[key] = fitted_model.conditional_volatility
    except Exception as e:
        print(f"Error fitting {key} model for primary benchmark {primary_benchmark_str}: {e}")

# Fit ARCH models for the secondary benchmark if it exists
for key, model in secondary_benchmark_arch_models.items():
    try:
        fitted_model = model.fit(disp="off")
        secondary_benchmark_arch_model_results[key] = fitted_model.conditional_volatility
    except Exception as e:
        print(f"Error fitting {key} model for secondary benchmark {secondary_benchmark_str}: {e}")

ticker_arch_model_results = pd.DataFrame(ticker_arch_model_results)
primary_benchmark_arch_model_results = pd.DataFrame(primary_benchmark_arch_model_results)
secondary_benchmark_arch_model_results = pd.DataFrame(secondary_benchmark_arch_model_results)

def plot_volatility(arch_model_result, returns_squared, title='Volatility and Squared Returns', threshold=3, show_spikes=True):
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    import numpy as np

    # Create subplot with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add volatility trace
    fig.add_trace(go.Scatter(
        x=arch_model_result.index,
        y=arch_model_result,
        mode='lines',
        name='Volatility',
        line=dict(color='cyan')
    ), secondary_y=False)

    if show_spikes:
        # Add squared returns trace
        fig.add_trace(go.Scatter(
            x=returns_squared.index,
            y=returns_squared,
            mode='lines', 
            name='Squared Returns',
            line=dict(color='orange')
        ), secondary_y=True)

        # Find spike dates for both thresholds
        spike_mean = returns_squared.mean()
        spike_std = returns_squared.std()
        
        threshold_value = spike_mean + threshold * spike_std
        spike_dates = returns_squared[returns_squared > threshold_value].index

        # Add vertical lines for threshold spikes
        for date in spike_dates:
            fig.add_shape(
                type="line",
                x0=date,
                x1=date,
                y0=0,
                y1=1,
                yref="paper",
                line=dict(
                    color="red",
                    width=1,
                    dash='longdash' # the options are 'solid', 'dot', 'dash', 'longdash', 'dashdot', or 'longdashdot'
                ),
                name=f"Spike ({threshold} SD)"
            )

    # Add a horizontal line for the median of volatility
    median_vol = arch_model_result.median()
    fig.add_hline(
        y=median_vol,
        line_dash="dot",
        line_color="yellow",
        annotation_text="Median",
        annotation_position="top left",
        secondary_y=False
    )

    # Calculate median absolute deviation (MAD) and add +- 1 MAD lines
    mad_vol = np.median(np.abs(arch_model_result - median_vol))
    fig.add_hline(
        y=median_vol + mad_vol,
        line_dash="dot",
        line_color="purple",
        annotation_text="+1 MAD",
        annotation_position="top left",
        secondary_y=False
    )
    fig.add_hline(
        y=median_vol - mad_vol,
        line_dash="dot",
        line_color="purple",
        annotation_text="-1 MAD",
        annotation_position="bottom left",
        secondary_y=False
    )

    # Add +- 2 MAD lines
    fig.add_hline(
        y=median_vol + 2 * mad_vol,
        line_dash="dot",
        line_color="green",
        annotation_text="+2 MAD",
        annotation_position="top left",
        secondary_y=False
    )
    fig.add_hline(
        y=median_vol - 2 * mad_vol,
        line_dash="dot",
        line_color="green",
        annotation_text="-2 MAD",
        annotation_position="bottom left",
        secondary_y=False
    )

    # Add -1.5 MAD line
    fig.add_hline(
        y=median_vol - 1.5 * mad_vol,
        line_dash="dot",
        line_color="red",
        annotation_text="-1.5 MAD",
        annotation_position="bottom left",
        secondary_y=False
    )

            # Add purple shading between -1.5 MAD and -1 MAD
    fig.add_shape(
        type="rect",
        x0=arch_model_result.index[0],
        x1=arch_model_result.index[-1],
        y0=median_vol - 1.5 * mad_vol,
        y1=median_vol - 1 * mad_vol,
        fillcolor= "purple",   
        opacity=0.2,
        line_width=0
    )

    fig.add_shape(
        type="rect",
        x0=arch_model_result.index[0],
        x1=arch_model_result.index[-1],
        y0=median_vol - 2 * mad_vol,
        y1=median_vol - 1.5 * mad_vol,
        fillcolor= "red",   
        opacity=0.2,
        line_width=0
    )

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title='Date',
        template='plotly_dark',
        height=1000,
        showlegend=True
    )
    
    fig.update_yaxes(title_text="Volatility", secondary_y=False)
    if show_spikes:
        fig.update_yaxes(title_text="Squared Returns", secondary_y=True)
    fig.show()


# Example usage for the main ticker
if 'egarch' in ticker_arch_model_results.columns:
    plot_volatility(
        ticker_arch_model_results['egarch'],
        squared_returns,
        title=f'EGARCH Model Volatility and Squared Returns for {ticker_str}',
        threshold=.5,
        show_spikes=False
    )

'''

'\nimport matplotlib.pyplot as plt\nplt.rcParams["figure.figsize"] = (40,14)\nfrom arch import arch_model\nimport yfinance as yf\n\n\nreturns = yf.Ticker(ticker_str).history(period=period, interval=interval)[\'Close\'].pct_change().dropna()\n\n# Calculate primary benchmark returns if available\nif primary_benchmark_str is not None:\n    primary_benchmark_returns = yf.Ticker(primary_benchmark_str).history(period=period, interval=interval)[\'Close\'].pct_change().dropna()\nelse:\n    primary_benchmark_returns = None\n\n# Calculate secondary benchmark returns if available\nif secondary_benchmark_str is not None:\n    secondary_benchmark_returns = yf.Ticker(secondary_benchmark_str).history(period=period, interval=interval)[\'Close\'].pct_change().dropna()\nelse:\n    secondary_benchmark_returns = None\n    \nticker_arch_models = {}\n\n# Define ARCH models for the main ticker\nticker_arch_models[\'garch\']      = arch_model(returns)\nticker_arch_models[\'egarch\']     = arch_model(returns, 

In [138]:
#VOLATILITY FORECASTING: Vix fix

#create a function that takes in a series of data and returns the vix fix
# the vix fix is a measure of the volatility premium
# it was created by Larry Williams

def vix_fix(series, window=22):
    """
    Computes the VIX Fix (Larry Williams) for a given price series.

    Parameters:
    - series (pd.Series): The input time series of prices.
    - window (int): The number of periods to look back for the highest high.

    Returns:
    - pd.Series: VIX Fix values.
    """
    highest_close = series.rolling(window=window).max()
    vix_fix_values = 100 * (highest_close - series) / highest_close
    return vix_fix_values

def plot_series_with_stdev_bands(
    data_series,
    stdev_values=[-0.5, 0.5, 1.5, 3],
    title="Series with Mean & Standard Deviations"
):
    """
    Plots a given data series, adding horizontal lines for mean and multiple standard deviations.
    Shades the regions between standard deviation bands with distinct colors:
    - Between -0.5 and 0.5 standard deviations: Green
    - Between 0.5 and 1.5 standard deviations: Yellow
    - Between 1.5 and 3 standard deviations: Red

    Parameters:
    - data_series (pd.Series): Any precomputed series of values to plot.
    - stdev_values (list of float): Multipliers for standard deviations to plot (e.g., [-0.5, 0.5, 1.5, 3]).
    - title (str): Chart title.
    """
    fig = go.Figure()

    # Plot data_series
    fig.add_trace(go.Scatter(
        x=data_series.index,
        y=data_series,
        mode='lines',
        name='Data',
        line=dict(color='yellow')
    ))

    # Compute mean and std
    mean_val = data_series.mean()
    std_val = data_series.std()

    # Add horizontal line for mean
    fig.add_hline(
        y=mean_val,
        line_color="white",
        line_dash="dash",
        annotation_text=f"Mean: {mean_val:.2f}",
        annotation_position="bottom right"
    )

    # Add horizontal lines for each standard deviation
    for stdev in stdev_values:
        sd_line = mean_val + stdev * std_val
        fig.add_hline(
            y=sd_line,
            line_color="white",
            line_dash="dot",
            annotation_text=f"{stdev} SD: {sd_line:.2f}",
            annotation_position="bottom right"
        )

    # Define colors for each shading band
    shade_colors = [
        "rgba(0, 255, 0, 0.3)",    # Green for -0.5 to 0.5
        "rgba(255, 255, 0, 0.5)",  # Yellow for 0.5 to 1.5
        "rgba(255, 0, 0, 0.7)"     # Red for 1.5 to 3
    ]

    # Sort stdev_values for consistent shading
    stdev_values_sorted = sorted(stdev_values)

    # Shade regions between consecutive standard deviation bands
    for i in range(len(stdev_values_sorted) - 1):
        lower_stdev = stdev_values_sorted[i]
        upper_stdev = stdev_values_sorted[i + 1]
        y0 = mean_val + lower_stdev * std_val
        y1 = mean_val + upper_stdev * std_val
        color = shade_colors[i] if i < len(shade_colors) else "rgba(255, 0, 0, 0.7)"
        
        fig.add_shape(
            type="rect",
            xref="x",
            yref="y",
            x0=data_series.index.min(),
            y0=y0,
            x1=data_series.index.max(),
            y1=y1,
            fillcolor=color,
            layer="below",
            line_width=0
        )
        
    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title='Value',
        template='plotly_dark',
        height=1000,
        xaxis=dict(
            # Set range to latest year only
            range=[
                data_series.index[-1] - pd.DateOffset(years=1), 
                data_series.index[-1]
            ]
        ),
        # Set y-axis range based on data from latest year only
        yaxis=dict(
            range=[
                data_series.min(),  # Minimum value of the data series
                data_series.max()   # Maximum value of the data series
            ]
        ),
    )

    fig.show()

def plot_series_with_stdev_bands(
    data_series,
    stdev_values=[-0.5, 0.5, 1.5, 3],
    num_years=5,
    title="Series with Mean & Standard Deviations"
):
    """
    Plots a given data series, adding horizontal lines for mean and multiple standard deviations.
    Shades the regions between standard deviation bands with distinct colors:
    - Between -0.5 and 0.5 standard deviations: Green
    - Between 0.5 and 1.5 standard deviations: Yellow
    - Between 1.5 and 3 standard deviations: Red

    Parameters:
    - data_series (pd.Series): Any precomputed series of values to plot.
    - stdev_values (list of float): Multipliers for standard deviations to plot (e.g., [-0.5, 0.5, 1.5, 3]).
    - num_years (int): Number of years to zoom in on the chart.
    - title (str): Chart title.
    """
    # Filter data to the specified number of years
    zoom_start = data_series.index[-1] - pd.DateOffset(years=num_years)
    zoom_data = data_series.loc[data_series.index >= zoom_start]

    fig = go.Figure()

    # Plot data_series
    fig.add_trace(go.Scatter(
        x=data_series.index,
        y=data_series,
        mode='lines',
        name='Data',
        line=dict(color='yellow')
    ))

    # Compute mean and std
    mean_val = data_series.mean()
    std_val = data_series.std()

    # Add horizontal line for mean
    fig.add_hline(
        y=mean_val,
        line_color="white",
        line_dash="dash",
        annotation_text=f"Mean: {mean_val:.2f}",
        annotation_position="bottom right"
    )

    # Add horizontal lines for each standard deviation
    for stdev in stdev_values:
        sd_line = mean_val + stdev * std_val
        fig.add_hline(
            y=sd_line,
            line_color="white",
            line_dash="dot",
            annotation_text=f"{stdev} SD: {sd_line:.2f}",
            annotation_position="bottom right"
        )

    # Define colors for each shading band
    shade_colors = [
        "rgba(0, 255, 0, 0.3)",    # Green for -0.5 to 0.5
        "rgba(255, 255, 0, 0.5)",  # Yellow for 0.5 to 1.5
        "rgba(255, 0, 0, 0.7)"     # Red for 1.5 to 3
    ]

    # Sort stdev_values for consistent shading
    stdev_values_sorted = sorted(stdev_values)

    # Shade regions between consecutive standard deviation bands
    for i in range(len(stdev_values_sorted) - 1):
        lower_stdev = stdev_values_sorted[i]
        upper_stdev = stdev_values_sorted[i + 1]
        y0 = mean_val + lower_stdev * std_val
        y1 = mean_val + upper_stdev * std_val
        color = shade_colors[i] if i < len(shade_colors) else "rgba(255, 0, 0, 0.7)"

        fig.add_shape(
            type="rect",
            xref="x",
            yref="y",
            x0=data_series.index.min(),
            y0=y0,
            x1=data_series.index.max(),
            y1=y1,
            fillcolor=color,
            layer="below",
            line_width=0
        )

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title='Value',
        template='plotly_dark',
        height=800,
        xaxis=dict(
            # Set range to the latest num_years only
            range=[zoom_start, data_series.index[-1]]
        ),
        # Adjust y-axis range based on filtered data
        yaxis=dict(
            range=[zoom_data.min(), zoom_data.max()]
        ),
    )

    fig.show()   
# ...existing code...
ticker_vix_fix = vix_fix(ticker['Close'])
if primary_benchmark_str is not None:
    primary_benchmark_vix_fix = vix_fix(primary_benchmark['Close'])
if secondary_benchmark_str is not None:
    secondary_benchmark_vix_fix = vix_fix(secondary_benchmark['Close'])


plot_series_with_stdev_bands(
    ticker_vix_fix,
    title='VIX Fix with Mean and Standard Deviations',
    stdev_values=[-0.5, 0.5, 1.5, 3]
)
'''
if primary_benchmark_str is not None:
    primary_benchmark_vix_fix = vix_fix(primary_benchmark['Close'])
    plot_series_with_stdev_bands(
        primary_benchmark_vix_fix,
        title=f'Benchmark 1 ({primary_benchmark_str}) VIX Fix with Mean and Standard Deviations',
        stdev_values=[-0.5, 0.5, 1.5, 3]
    )

if secondary_benchmark_str is not None:
    secondary_benchmark_vix_fix = vix_fix(secondary_benchmark['Close'])
    plot_series_with_stdev_bands(
        secondary_benchmark_vix_fix,
        title=f'Benchmark 2 ({secondary_benchmark_str}) VIX Fix with Mean and Standard Deviations',
        stdev_values=[-0.5, 0.5, 1.5, 3]
    )'''

"\nif primary_benchmark_str is not None:\n    primary_benchmark_vix_fix = vix_fix(primary_benchmark['Close'])\n    plot_series_with_stdev_bands(\n        primary_benchmark_vix_fix,\n        title=f'Benchmark 1 ({primary_benchmark_str}) VIX Fix with Mean and Standard Deviations',\n        stdev_values=[-0.5, 0.5, 1.5, 3]\n    )\n\nif secondary_benchmark_str is not None:\n    secondary_benchmark_vix_fix = vix_fix(secondary_benchmark['Close'])\n    plot_series_with_stdev_bands(\n        secondary_benchmark_vix_fix,\n        title=f'Benchmark 2 ({secondary_benchmark_str}) VIX Fix with Mean and Standard Deviations',\n        stdev_values=[-0.5, 0.5, 1.5, 3]\n    )"

In [139]:
#VOLATILITY FORECASTING: VIX 
# the volatility premium is the difference between the implied volatility and the realized volatility
# the implied volatility is the expected volatility of the market in the future
# the realized volatility is the actual volatility of the market in the past

# The volatility premium is a measure of the market's fear and uncertainty
# the greater the volatility premium, the greate the reward we get for taking on the risk of the market
# the lower the volatility premium, the lower the reward we get for taking on the risk of the market

# when trading options, we want to buy options when volatility is low and sell options when volatility is high
# we can use the volatility premium to tell us how much reward we should take on for the risk we are taking on
# we can also trade in terms of time periods, 
# for example

# when options when volatility is low  we should sell options with a long time period to expiration
# when volatility is high, we should buy options with a short time period to expiration

# In trading, high volatility is a sign for opportunity, and should be taken advantage of
# low volatility is a sign for stability, and a low opportunity for profit
# thus we wait for high volatility to enter the market, and slowly exit the market when volatility is low

# but it also tell us how to scale our positions
# when volatility is high, we should scale our positions down
# when volatility is low, we should scale our positions up


#Rules for trading the volatility premium ( a bit sophisticated ) choose the smallest time frame where vix is greater than the moving average
# 1. When Vix is greater than 21 day moving average, trade SPY options at 21 days to expiration
# 2. When Vix is greater than 50 day moving average, trade SPY options at 50 days to expiration
# 3. When Vix is greater than 200 day moving average, trade SPY options at 200 days to expiration

# the lowest time frame where vix is greater than the moving average is the time frame we should trade options at

#SPECIAL CASE *******************************************************************************************
# # IF WE ARE IN A NEUTRAL REGIME ##                                                                    *
# SHORT HEDGE THE POSITION ACCORDING:                                                                   *
# 1. pick the simple moving average that is smallest among the 21, 50, and 200 day moving averages      *
#                                                                                                       *
# The reason why we do this is because in most cases we expect regime changes to be tempororary         *
# and we want to take advantage of these regime changes instead of waiting for the market to switch back*
# to bullish. BUT. somtimes (as in the case of bear markets) regimes dont change quickly and can persist*
# thus we tie try to capitalize on these momements where the regime is persitent                        *
#********************************************************************************************************

#for example, if vix is greater than the 21 day moving average and the lowest absolute value is 200 days 
# we should trade options at 21 days and hedge that at 200 days

'''

period = '5y'  # Last 2 years to ensure moving averages are calculated correctly
interval = '1d'  # Example interval

# Download VIX data
vix_ticker = yf.Ticker('^VIX')
vix_data = vix_ticker.history(period=period, interval=interval)
vix_data_short = vix_ticker.history(period='1y', interval=interval)

#short term moving averages
# Calculate the 21-day, 50-day, and 200-day Simple Moving Averages (SMAs)

vix_data_short['9_'+'SMA'] = vix_data_short['Close'].rolling(window=9).mean()
vix_data_short['13_'+'SMA'] = vix_data_short['Close'].rolling(window=13).mean()
vix_data_short['20_'+'SMA'] = vix_data_short['Close'].rolling(window=20).mean()

#long term moving averages
# Calculate the 21-day, 50-day, and 200-day Simple Moving Averages (SMAs)
vix_data['21_SMA'] = vix_data['Close'].rolling(window=21).mean()
vix_data['50_SMA'] = vix_data['Close'].rolling(window=50).mean()
vix_data['200_SMA'] = vix_data['Close'].rolling(window=200).mean()

    

def plot_vix_with_sma(vix_data, title='VIX Candlestick Chart with SMAs and Trading Annotations'):
    """
    Plots the VIX candlestick chart with all SMAs present in the DataFrame,
    and adds annotations indicating which time frame to trade and hedge based on the strategy.

    Parameters:
        vix_data (pd.DataFrame): DataFrame containing VIX data with calculated SMAs.
        title (str): Title of the plot (default is 'VIX Candlestick Chart with SMAs and Trading Annotations').
    """
    import plotly.graph_objects as go

    # Create the Plotly figure
    fig = go.Figure()

    # Add VIX candlestick data
    fig.add_trace(go.Candlestick(
        x=vix_data.index,
        open=vix_data['Open'],
        high=vix_data['High'],
        low=vix_data['Low'],
        close=vix_data['Close'],
        name='VIX'
    ))

    # Add SMAs to the figure
    sma_columns = [col for col in vix_data.columns if 'SMA' in col]
    sma_colors = ['blue', 'green', 'orange', 'red', 'purple', 'yellow']
    for i, sma_column in enumerate(sma_columns):
        fig.add_trace(go.Scatter(
            x=vix_data.index,
            y=vix_data[sma_column],
            mode='lines',
            name=sma_column,
            line=dict(color=sma_colors[i % len(sma_colors)])
        ))

    # Determine the time frame to trade
    current_vix = vix_data['Close'].iloc[-1]
    sma_values = {int(col.split('_')[0]): vix_data[col].iloc[-1] for col in sma_columns}
    eligible_time_frames = [tf for tf, sma in sma_values.items() if current_vix > sma]

    if eligible_time_frames:
        trade_time_frame = min(eligible_time_frames)
        trade_action = f"Trade SPY options with {trade_time_frame}-day expiration"
    else:
        trade_action = "No trade signal"

    # Determine the hedge time frame based on the smallest SMA value
    smallest_sma_timeframe = min(sma_values, key=sma_values.get)
    hedge_action = f"if in neutral regime, Hedge with {smallest_sma_timeframe}-day options"

    # Combine the annotations
    annotation_text = f"{trade_action}<br>{hedge_action}"

    # Add annotation to the plot
    fig.add_annotation(
        x=vix_data.index[-1],
        y=current_vix,
        text=annotation_text,
        showarrow=True,
        arrowhead=1,
        ax=-70,
        ay=-70,
        bgcolor='black',
        font=dict(color='white')
    )

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title='VIX Value',
        height=800,
        template='plotly_dark'
    )

    fig.show()
def plot_vix_difference_sma_list(vix_data, windows = [21, 50, 200]):
    """
    Plots the difference between the VIX Close price and its SMAs for multiple window sizes.

    Parameters:
        vix_data (pd.DataFrame): DataFrame containing VIX data.
        windows (list): List of window sizes for SMA calculations.
    """

    # Create subplots with shared x-axis
    fig = sp.make_subplots(
        rows=len(windows),
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.02,
        subplot_titles=[f'VIX - {window}-Day SMA' for window in windows]
    )

    # For each window size, calculate SMA and plot the difference
    for i, window in enumerate(windows):
        sma_column = f'{window}_SMA'
        diff_column = f'Diff_{window}'

        # Calculate SMA
        vix_data[sma_column] = vix_data['Close'].rolling(window=window).mean()
        # Calculate difference
        vix_data[diff_column] = vix_data['Close'] - vix_data[sma_column]

        # Add difference line to subplot
        fig.add_trace(
            go.Scatter(
                x=vix_data.index,
                y=vix_data[diff_column],
                mode='lines',
                name=f'VIX - {window}-Day SMA'
            ),
            row=i + 1,
            col=1
        )

        # Add horizontal line at y=0
        fig.add_trace(
            go.Scatter(
                x=vix_data.index,
                y=[0] * len(vix_data),
                mode='lines',
                line=dict(color='red', dash='dash'),
                showlegend=False
            ),
            row=i + 1,
            col=1
        )

    # Update layout
    fig.update_layout(
        height=300 * len(windows),
        title='Difference Between VIX and SMAs',
        template='plotly_dark',
        showlegend=False
    )

    # Update x-axis labels
    fig.update_xaxes(title_text='Date', row=len(windows), col=1)
    # Update y-axis labels
    for i in range(len(windows)):
        fig.update_yaxes(title_text='Difference', row=i + 1, col=1)

    fig.show()

# Call the function to plot the data
plot_vix_with_sma(vix_data, title='VIX Candlestick Chart with 21-Day, 50-Day, and 200-Day SMAs and Trading Annotations')
plot_vix_difference_sma_list(vix_data)
'''

'\n\nperiod = \'5y\'  # Last 2 years to ensure moving averages are calculated correctly\ninterval = \'1d\'  # Example interval\n\n# Download VIX data\nvix_ticker = yf.Ticker(\'^VIX\')\nvix_data = vix_ticker.history(period=period, interval=interval)\nvix_data_short = vix_ticker.history(period=\'1y\', interval=interval)\n\n#short term moving averages\n# Calculate the 21-day, 50-day, and 200-day Simple Moving Averages (SMAs)\n\nvix_data_short[\'9_\'+\'SMA\'] = vix_data_short[\'Close\'].rolling(window=9).mean()\nvix_data_short[\'13_\'+\'SMA\'] = vix_data_short[\'Close\'].rolling(window=13).mean()\nvix_data_short[\'20_\'+\'SMA\'] = vix_data_short[\'Close\'].rolling(window=20).mean()\n\n#long term moving averages\n# Calculate the 21-day, 50-day, and 200-day Simple Moving Averages (SMAs)\nvix_data[\'21_SMA\'] = vix_data[\'Close\'].rolling(window=21).mean()\nvix_data[\'50_SMA\'] = vix_data[\'Close\'].rolling(window=50).mean()\nvix_data[\'200_SMA\'] = vix_data[\'Close\'].rolling(window=200).mea

In [140]:
#SEASONALITY: Monthly Seasonality

fig_ticker_Seasonality_Monthly = qp.plot_seasonality(ticker_monthly_returns, f'Monthly Seasonality: {ticker_str}', 'monthly')
fig_ticker_Seasonality_Monthly.show()
'''
if primary_benchmark_str is not None:
    fig_primary_benchmark_Seasonality_Monthly = qp.plot_seasonality(primary_benchmark_monthly_returns, f'Monthly Seasonality: {primary_benchmark_str}', 'monthly')
    fig_primary_benchmark_Seasonality_Monthly.show()

if secondary_benchmark_str is not None:
    fig_secondary_benchmark_Seasonality_Monthly = qp.plot_seasonality(secondary_benchmark_monthly_returns, f'Monthly Seasonality: {secondary_benchmark_str}', 'monthly')
    fig_secondary_benchmark_Seasonality_Monthly.show()
'''

"\nif primary_benchmark_str is not None:\n    fig_primary_benchmark_Seasonality_Monthly = qp.plot_seasonality(primary_benchmark_monthly_returns, f'Monthly Seasonality: {primary_benchmark_str}', 'monthly')\n    fig_primary_benchmark_Seasonality_Monthly.show()\n\nif secondary_benchmark_str is not None:\n    fig_secondary_benchmark_Seasonality_Monthly = qp.plot_seasonality(secondary_benchmark_monthly_returns, f'Monthly Seasonality: {secondary_benchmark_str}', 'monthly')\n    fig_secondary_benchmark_Seasonality_Monthly.show()\n"

In [141]:
#SEASONALITY: Weekly Seasonality
fig_ticker_Seasonality_weekly = qp.plot_seasonality(ticker_weekly_returns, f'Weekly Seasonality: {ticker_str}', 'weekly')
fig_ticker_Seasonality_weekly.show()
'''
if primary_benchmark_str is not None:
    fig_primary_benchmark_Seasonality_weekly = qp.plot_seasonality(primary_benchmark_weekly_returns, f'Weekly Seasonality: {primary_benchmark_str}', 'weekly')
    fig_primary_benchmark_Seasonality_weekly.show()

if secondary_benchmark_str is not None:
    fig_secondary_benchmark_Seasonality_weekly = qp.plot_seasonality(secondary_benchmark_weekly_returns, f'Weekly Seasonality: {secondary_benchmark_str}', 'weekly')
    fig_secondary_benchmark_Seasonality_weekly.show()
    
'''

"\nif primary_benchmark_str is not None:\n    fig_primary_benchmark_Seasonality_weekly = qp.plot_seasonality(primary_benchmark_weekly_returns, f'Weekly Seasonality: {primary_benchmark_str}', 'weekly')\n    fig_primary_benchmark_Seasonality_weekly.show()\n\nif secondary_benchmark_str is not None:\n    fig_secondary_benchmark_Seasonality_weekly = qp.plot_seasonality(secondary_benchmark_weekly_returns, f'Weekly Seasonality: {secondary_benchmark_str}', 'weekly')\n    fig_secondary_benchmark_Seasonality_weekly.show()\n    \n"

In [142]:
#SEASONALITY: Daily Seasonality
fig_ticker_Seasonality_daily = qp.plot_seasonality(ticker_daily_returns, f'Daily Seasonality: {ticker_str}', 'daily')
fig_ticker_Seasonality_daily.show()
'''
if primary_benchmark_str is not None:
    fig_primary_benchmark_Seasonality_daily = qp.plot_seasonality(primary_benchmark_daily_returns, f'Daily Seasonality: {primary_benchmark_str}', 'daily')
    fig_primary_benchmark_Seasonality_daily.show()
    

if secondary_benchmark_str is not None:
    fig_secondary_benchmark_Seasonality_daily = qp.plot_seasonality(secondary_benchmark_daily_returns, f'Daily Seasonality: {secondary_benchmark_str}', 'daily')
    fig_secondary_benchmark_Seasonality_daily.show()
'''

"\nif primary_benchmark_str is not None:\n    fig_primary_benchmark_Seasonality_daily = qp.plot_seasonality(primary_benchmark_daily_returns, f'Daily Seasonality: {primary_benchmark_str}', 'daily')\n    fig_primary_benchmark_Seasonality_daily.show()\n    \n\nif secondary_benchmark_str is not None:\n    fig_secondary_benchmark_Seasonality_daily = qp.plot_seasonality(secondary_benchmark_daily_returns, f'Daily Seasonality: {secondary_benchmark_str}', 'daily')\n    fig_secondary_benchmark_Seasonality_daily.show()\n"

In [143]:
#Calculations


sharpe_ticker_mid = qc.calculate_risk_adjusted_returns(ticker['Close'], ratio_type='sharpe', windows=[time_frame_mid])['sharpe_ratio_50']
sharpe_ticker_long = qc.calculate_risk_adjusted_returns(ticker['Close'], ratio_type='sharpe', windows=[time_frame_long])[f'sharpe_ratio_200']



'''
spread_mid_primary = primary_benchmark['Close'].pct_change(time_frame_mid) - ticker['Close'].pct_change(time_frame_mid)
spread_long_primary = primary_benchmark['Close'].pct_change(time_frame_long) - ticker['Close'].pct_change(time_frame_long)
sharpe_benchmark_mid_primary = qc.calculate_risk_adjusted_returns(primary_benchmark['Close'], ratio_type='sharpe', windows=[time_frame_mid])['sharpe_ratio_50']
sharpe_benchmark_long_primary = qc.calculate_risk_adjusted_returns(primary_benchmark['Close'], ratio_type='sharpe', windows=[time_frame_long])['sharpe_ratio_200']
sharpe_spread_mid_primary = sharpe_benchmark_mid_primary - sharpe_ticker_mid
sharpe_spread_long_primary = sharpe_benchmark_long_primary - sharpe_ticker_long


spread_mid_secondary = secondary_benchmark['Close'].pct_change(time_frame_mid) - ticker['Close'].pct_change(time_frame_mid)
spread_long_secondary = secondary_benchmark['Close'].pct_change(time_frame_long) - ticker['Close'].pct_change(time_frame_long)
sharpe_benchmark_mid_secondary = qc.calculate_risk_adjusted_returns(secondary_benchmark['Close'], ratio_type='sharpe', windows=[time_frame_mid])['sharpe_ratio_50']
sharpe_benchmark_long_secondary = qc.calculate_risk_adjusted_returns(secondary_benchmark['Close'], ratio_type='sharpe', windows=[time_frame_long])['sharpe_ratio_200']
sharpe_spread_mid_secondary = sharpe_benchmark_mid_secondary - sharpe_ticker_mid
sharpe_spread_long_secondary= sharpe_benchmark_long_secondary - sharpe_ticker_long'''

if primary_benchmark is not None:
    spread_mid_primary = primary_benchmark['Close'].pct_change(time_frame_mid) - ticker['Close'].pct_change(time_frame_mid)
    spread_long_primary = primary_benchmark['Close'].pct_change(time_frame_long) - ticker['Close'].pct_change(time_frame_long)
    
    sharpe_benchmark_mid_primary = qc.calculate_risk_adjusted_returns(
        primary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_mid]
    )['sharpe_ratio_50']
    
    sharpe_benchmark_long_primary = qc.calculate_risk_adjusted_returns(
        primary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_long]
    )['sharpe_ratio_200']
    
    sharpe_spread_mid_primary = sharpe_benchmark_mid_primary - sharpe_ticker_mid
    sharpe_spread_long_primary = sharpe_benchmark_long_primary - sharpe_ticker_long

# Calculate spreads and Sharpe ratios for the secondary benchmark if it exists
if secondary_benchmark is not None:
    spread_mid_secondary = secondary_benchmark['Close'].pct_change(time_frame_mid) - ticker['Close'].pct_change(time_frame_mid)
    spread_long_secondary = secondary_benchmark['Close'].pct_change(time_frame_long) - ticker['Close'].pct_change(time_frame_long)
    
    sharpe_benchmark_mid_secondary = qc.calculate_risk_adjusted_returns(
        secondary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_mid]
    )['sharpe_ratio_50']
    
    sharpe_benchmark_long_secondary = qc.calculate_risk_adjusted_returns(
        secondary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_long]
    )['sharpe_ratio_200']
    
    sharpe_spread_mid_secondary = sharpe_benchmark_mid_secondary - sharpe_ticker_mid
    sharpe_spread_long_secondary = sharpe_benchmark_long_secondary - sharpe_ticker_long



In [144]:
#RISK ADJUSTED RETURNS: Mid Term
# Plot for primary benchmark if it exists

if primary_benchmark is not None:
    qp.create_spread_plot(
        spread_mid_primary,
        title=f'{time_frame_mid} Day: Benchmark ({primary_benchmark_str}) minus {ticker_str}',
        default_years=5
    ).show()
    
    qp.create_spread_plot(
        sharpe_spread_mid_primary,
        title=f'{time_frame_mid} Day: Benchmark ({primary_benchmark_str}) minus {ticker_str} (Sharpe)',
        default_years=10
    ).show()
    
    qp.plot_returns(
        ticker['Close'],
        benchmark_series=primary_benchmark['Close'],
        plot_type='risk_adjusted',
        title=f'{ticker_str} Returns {time_frame_mid} days',
        window=time_frame_mid
    ).show()

# Plot for secondary benchmark if it exists
if secondary_benchmark is not None:
    qp.create_spread_plot(
        spread_mid_secondary,
        title=f'{time_frame_mid} Day: Benchmark ({secondary_benchmark_str}) minus {ticker_str}',
        default_years=5
    ).show()
    
    qp.create_spread_plot(
        sharpe_spread_mid_secondary,
        title=f'{time_frame_mid} Day: Benchmark ({secondary_benchmark_str}) minus {ticker_str} (Sharpe)',
        default_years=10
    ).show()
    
    qp.plot_returns(
        ticker['Close'],
        benchmark_series=secondary_benchmark['Close'],
        plot_type='risk_adjusted',
        title=f'{ticker_str} Returns {time_frame_mid} days',
        window=time_frame_mid
    ).show()
  

In [145]:
#RISK ADJUSTED RETURNS: Long Term

if primary_benchmark is not None:
    qp.create_spread_plot(
        spread_long_primary,
        title=f'{time_frame_long} Day: Benchmark ({primary_benchmark_str}) minus {ticker_str}',
        default_years=5
    ).show()
    
    qp.create_spread_plot(
        sharpe_spread_long_primary,
        title=f'{time_frame_long} Day: Benchmark ({primary_benchmark_str}) minus {ticker_str} (Sharpe)',
        default_years=10
    ).show()
    
    qp.plot_returns(
        ticker['Close'],
        benchmark_series=primary_benchmark['Close'],
        plot_type='risk_adjusted',
        title=f'{ticker_str} Returns {time_frame_long} days',
        window=time_frame_long
    ).show()

# Plot for secondary benchmark if it exists
if secondary_benchmark is not None:
    qp.create_spread_plot(
        spread_long_secondary,
        title=f'{time_frame_long} Day: Benchmark ({secondary_benchmark_str}) minus {ticker_str}',
        default_years=5
    ).show()
    
    qp.create_spread_plot(
        sharpe_spread_long_secondary,
        title=f'{time_frame_long} Day: Benchmark ({secondary_benchmark_str}) minus {ticker_str} (Sharpe)',
        default_years=10
    ).show()
    
    qp.plot_returns(
        ticker['Close'],
        benchmark_series=secondary_benchmark['Close'],
        plot_type='risk_adjusted',
        title=f'{ticker_str} Returns {time_frame_long} days',
        window=time_frame_long
    ).show()
    

In [146]:
'''
#Sector and Sub-Industry Analysis

#this returns a dataframe with columns 'Symbol', 'Sector', 'Sub-Industry
sp500_companies = qe.retrieve_market_data()['SP500']

# given ticker_str, return the Sector and Sub-Industry where Symbol = ticker_str
ticker_info = sp500_companies[sp500_companies['Symbol'] == ticker_str]
ticker_sector = ticker_info['Sector'].values[0]
ticker_sub_industry = ticker_info['Sub-Industry'].values[0]

#give me the list of all companies in the same sector as ticker_str
sector_companies = sp500_companies[sp500_companies['Sector'] == ticker_sector]

#given ticker_str, return the list of all companies in the same sub-industry as ticker_str
sub_industry_companies = sp500_companies[sp500_companies['Sub-Industry'] == ticker_sub_industry]

#print the sector and sub-industry of the ticker_str
print('Sector of {}: {}'.format(ticker_str, ticker_sector))
print('Sub-Industry of {}: {}'.format(ticker_str, ticker_sub_industry))

print('Companies in the same sector as {}: {}'.format(ticker_str, sector_companies['Symbol'].values))
print('Companies in the same sub-industry as {}: {}'.format(ticker_str, sub_industry_companies['Symbol'].values))   

#create a function that takes in a list of tickers and returns a dataframe with the closing prices of the tickers

def get_closing_prices(tickers, period=period, interval=interval):
    """
    Retrieves the closing prices of the specified tickers over the specified period and interval.
    
    Parameters:
        tickers (list): A list of stock tickers to retrieve closing prices for.
        period (str): The period to retrieve data for (e.g., '1y', '5d', '10y').
        interval (str): The interval of the data (e.g., '1d', '1wk', '1mo').

    Returns:
        pd.DataFrame: A DataFrame containing the closing prices of the tickers.
    """
    data = pd.DataFrame()
    for ticker in tickers:
        t = yf.Ticker(ticker)
        hist = t.history(period=period, interval=interval)
        data[ticker] = hist['Close']
    return data
def calculate_rolling_returns(prices, time_frame, calculation_method='individual'):
    """
    Calculates the rolling returns of the specified stock prices over the specified time frame.
    
    Parameters:
        prices (pd.DataFrame): A DataFrame containing the stock prices.
        time_frame (int): The time frame over which to calculate the rolling returns.
        calculation_method (str): 'individual' to return individual stock rolling returns,
                                   'average' to return the average rolling returns across stocks.
    
    Returns:
        pd.DataFrame or pd.Series: A DataFrame containing the rolling returns of the stocks if 'individual',
                                   or a Series with the average rolling returns if 'average'.
    """
    returns = prices.pct_change()
    rolling_returns = returns.rolling(time_frame).sum()
    
    if calculation_method == 'average':
        return simplify_datetime_index(rolling_returns).mean(axis=1)
    else:
        return simplify_datetime_index(rolling_returns)
def calculate_rolling_risk_adjusted_returns(prices, time_frame, calculation_method='individual', ratio_type='sharpe'):
    """
    Calculates the rolling risk-adjusted returns of the specified stock prices over the specified time frame.
    
    Parameters:
        prices (pd.DataFrame): A DataFrame containing the stock prices.
        time_frame (int): The time frame over which to calculate the rolling risk-adjusted returns.
        calculation_method (str): 
            - 'individual' to return individual stock rolling risk-adjusted returns,
            - 'average' to return the average rolling risk-adjusted returns across stocks.
        ratio_type (str): The type of risk-adjusted return to calculate (only 'sharpe' is implemented).
    
    Returns:
        pd.DataFrame or pd.Series: 
            - A DataFrame containing the rolling risk-adjusted returns of the stocks if 'individual',
            - Or a Series with the average rolling risk-adjusted returns if 'average'.
    """
    if ratio_type != 'sharpe':
        raise NotImplementedError("Currently only 'sharpe' ratio_type is supported.")
    
    # Calculate daily returns
    returns = prices.pct_change()
    
    
    # Define risk-free rate (assumed to be 0 for simplicity)
    risk_free_rate = 0.0
    
    # Calculate excess returns
    excess_returns = returns - risk_free_rate
    # Calculate rolling mean of excess returns
    rolling_mean = excess_returns.rolling(window=time_frame).mean()
    
    
    # Calculate rolling standard deviation of returns
    rolling_std = returns.rolling(window=time_frame).std()
    
    # Calculate Sharpe ratio
    rolling_sharpe = rolling_mean / rolling_std
    
    
    # Simplify the datetime index
    rolling_sharpe = simplify_datetime_index(rolling_sharpe)
    
    if calculation_method == 'average':
        return rolling_sharpe.mean(axis=1)
    else:
        return rolling_sharpe

sector_prices = get_closing_prices(sector_companies['Symbol'].values)
sub_industry_prices = get_closing_prices(sub_industry_companies['Symbol'].values)

sector_rolling_returns      = calculate_rolling_returns(sector_prices, 1 ,calculation_method='individual')
sector_rolling_returns_short = calculate_rolling_returns(sector_prices, time_frame_short,calculation_method='individual')
sector_rolling_returns_mid = calculate_rolling_returns(sector_prices, time_frame_mid,calculation_method='individual')  
sector_rolling_returns_long = calculate_rolling_returns(sector_prices, time_frame_long,calculation_method='individual')

sector_average_rolling_returns = calculate_rolling_returns(sector_prices, 1 ,calculation_method='average')
sector_average_rolling_returns_short = calculate_rolling_returns(sector_prices, time_frame_short,calculation_method='average')
sector_average_rolling_returns_mid = calculate_rolling_returns(sector_prices, time_frame_mid,calculation_method='average')  
sector_average_rolling_returns_long = calculate_rolling_returns(sector_prices, time_frame_long,calculation_method='average')

sector_rolling_risk_adjusted_returns_short = calculate_rolling_risk_adjusted_returns(sector_prices, time_frame_short,calculation_method='individual')
sector_rolling_risk_adjusted_returns_mid = calculate_rolling_risk_adjusted_returns(sector_prices, time_frame_mid,calculation_method='individual')
sector_rolling_risk_adjusted_returns_long = calculate_rolling_risk_adjusted_returns(sector_prices, time_frame_long,calculation_method='individual')

sector_average_rolling_risk_adjusted_returns_short = calculate_rolling_risk_adjusted_returns(sector_prices, time_frame_short,calculation_method='average')
sector_average_rolling_risk_adjusted_returns_mid = calculate_rolling_risk_adjusted_returns(sector_prices, time_frame_mid,calculation_method='average')
sector_average_rolling_risk_adjusted_returns_long = calculate_rolling_risk_adjusted_returns(sector_prices, time_frame_long,calculation_method='average')



sub_industry_rolling_returns     = calculate_rolling_returns(sub_industry_prices, 1 ,calculation_method='individual')
sub_industry_rolling_returns_short = calculate_rolling_returns(sub_industry_prices, time_frame_short,calculation_method='individual')
sub_industry_rolling_returns_mid = calculate_rolling_returns(sub_industry_prices, time_frame_mid,calculation_method='individual')
sub_industry_rolling_returns_long = calculate_rolling_returns(sub_industry_prices, time_frame_long,calculation_method='individual')

sub_industry_average_rolling_returns = calculate_rolling_returns(sub_industry_prices, 1 ,calculation_method='average')
sub_industry_average_rolling_returns_short = calculate_rolling_returns(sub_industry_prices, time_frame_short,calculation_method='average')
sub_industry_average_rolling_returns_mid = calculate_rolling_returns(sub_industry_prices, time_frame_mid,calculation_method='average')
sub_industry_average_rolling_returns_long = calculate_rolling_returns(sub_industry_prices, time_frame_long,calculation_method='average')    

sub_industry_rolling_risk_adjusted_returns_short = calculate_rolling_risk_adjusted_returns(sub_industry_prices, time_frame_short,calculation_method='individual')
sub_industry_rolling_risk_adjusted_returns_mid = calculate_rolling_risk_adjusted_returns(sub_industry_prices, time_frame_mid,calculation_method='individual')
sub_industry_rolling_risk_adjusted_returns_long = calculate_rolling_risk_adjusted_returns(sub_industry_prices, time_frame_long,calculation_method='individual')

sub_industry_average_rolling_risk_adjusted_returns_short = calculate_rolling_risk_adjusted_returns(sub_industry_prices, time_frame_short,calculation_method='average')
sub_industry_average_rolling_risk_adjusted_returns_mid = calculate_rolling_risk_adjusted_returns(sub_industry_prices, time_frame_mid,calculation_method='average')
sub_industry_average_rolling_risk_adjusted_returns_long = calculate_rolling_risk_adjusted_returns(sub_industry_prices, time_frame_long,calculation_method='average')

fig = px.line(
    sector_rolling_returns_long,
    title=f"Sector({ticker_sector}) Rolling Returns (Long)"
)
fig.update_layout(width=1200, height=600)
fig.show()


fig = px.line(
    sub_industry_rolling_returns_long,
    title=f"Sub-industry({ticker_sub_industry}) Rolling Returns (Long)"
)
fig.update_layout(width=1200, height=600)
fig.show()


fig = px.line(
    sector_rolling_risk_adjusted_returns_long,
    title=f"Sector({ticker_sector}) Rolling Risk-Adjusted Returns (Long)"
)
fig.update_layout(width=1200, height=600)
fig.show()


fig = px.line(
    sub_industry_rolling_risk_adjusted_returns_long,
    title=f"Sub-industry({ticker_sub_industry}) Rolling Risk-Adjusted Returns (Long)"
)

fig.update_layout(width=1200, height=600)
fig.show()
'''

'\n#Sector and Sub-Industry Analysis\n\n#this returns a dataframe with columns \'Symbol\', \'Sector\', \'Sub-Industry\nsp500_companies = qe.retrieve_market_data()[\'SP500\']\n\n# given ticker_str, return the Sector and Sub-Industry where Symbol = ticker_str\nticker_info = sp500_companies[sp500_companies[\'Symbol\'] == ticker_str]\nticker_sector = ticker_info[\'Sector\'].values[0]\nticker_sub_industry = ticker_info[\'Sub-Industry\'].values[0]\n\n#give me the list of all companies in the same sector as ticker_str\nsector_companies = sp500_companies[sp500_companies[\'Sector\'] == ticker_sector]\n\n#given ticker_str, return the list of all companies in the same sub-industry as ticker_str\nsub_industry_companies = sp500_companies[sp500_companies[\'Sub-Industry\'] == ticker_sub_industry]\n\n#print the sector and sub-industry of the ticker_str\nprint(\'Sector of {}: {}\'.format(ticker_str, ticker_sector))\nprint(\'Sub-Industry of {}: {}\'.format(ticker_str, ticker_sub_industry))\n\nprint(\'Co